In [0]:
# %sql

# select * from dimruncycle
# order by runcycleid desc
# limit 1;


In [0]:
%sql

-- Drop the function if it already exists to allow recreation during development
DROP FUNCTION IF EXISTS udf_get_workdays_bc;

-- Create the SQL UDF
CREATE FUNCTION udf_get_workdays_bc(
    startDate DATE,
    endDate DATE,
    bcHolidays ARRAY<DATE> DEFAULT ARRAY() -- Optional parameter, defaults to an empty array
)
RETURNS INT
RETURN (
    CASE
        WHEN startDate IS NULL OR endDate IS NULL THEN 0
        WHEN endDate < startDate THEN 0
        ELSE
            REDUCE(
                -- 1. Generate a sequence of all dates between startDate and endDate
                SEQUENCE(startDate, endDate, INTERVAL 1 DAY),
                -- 2. Initialize the accumulator (workday count) to 0
                0,
                -- 3. Lambda function to process each date in the sequence
                (acc, current_date) -> acc + CASE
                                            -- Check if it's a weekend (Sunday=1, Saturday=7 in DAYOFWEEK)
                                            WHEN DAYOFWEEK(current_date) IN (1, 7) THEN 0
                                            -- Check if the current_date is in the provided bcHolidays array
                                            WHEN ARRAY_CONTAINS(bcHolidays, current_date) THEN 0
                                            -- If not a weekend and not a holiday, it's a workday
                                            ELSE 1
                                        END
            )
    END
);


In [0]:
# %sql

#         SELECT
#             0 as runcycleid,
#             mr.foirequest_id as foirequestid,
#             CASE
#                 WHEN r.requesttype = 'personal' THEN 33
#             ELSE
#                 31
#             END AS requesttypeid,
#             CASE
#                 WHEN mr.requeststatusid IS NOT NULL THEN mr.requeststatusid
#             ELSE
#                 (SELECT requeststatusid FROM foi_mod.foirequeststatuses WHERE name = rr.status LIMIT 1)
#             END AS requeststatusid,
#             TRY_CAST(r.receivedmodeid AS STRING) AS receivedmodeid,
#             TRY_CAST(r.deliverymodeid AS STRING) AS deliverymodeid,
#             r.applicantcategoryid,
#             TRY_TO_TIMESTAMP(r.receiveddate, 'yyyy-MM-dd HH:mm:ss') AS requesteddate,
#             TRY_TO_TIMESTAMP(r.receiveddate, 'yyyy-MM-dd HH:mm:ss') AS receiveddate,
#             TRY_TO_TIMESTAMP(mr.startdate, 'yyyy-MM-dd HH:mm:ss') AS startdate,
#             TRY_TO_TIMESTAMP(first_rawrequest.created_at, 'yyyy-MM-dd HH:mm:ss') AS createddate,
#             TRY_TO_TIMESTAMP(r.created_at, 'yyyy-MM-dd HH:mm:ss') AS modifieddate,
#             CASE
#                WHEN mr.assignedto IS NOT NULL THEN ministryassignedto.time_assignedto_changed_to_current_value
#             ELSE
#                 rawassignedto.time_assignedto_changed_to_current_value
#             END AS assigneddate,
#             CAST(NULL AS STRING) AS receivedbyusername,
#             r.createdby AS modifiedbyusername,
#             CAST(NULL AS INTEGER) AS assignedbyid,
#             CAST(NULL AS INTEGER) AS assignedtoid,
#             CASE
#                 WHEN mr.assignedto IS NOT NULL THEN mr.assignedto 
#             ELSE
#                 rr.assignedto
#             END AS primaryusername,
#             0 AS primarygroupid,
#             CASE
#                 WHEN mr.assignedgroup IS NOT NULL THEN mr.assignedgroup 
#             ELSE
#                 rr.assignedgroup
#             END AS groupname,
#             office.bcgovcode AS officeid,
#             mr.assignedministrygroup AS ministry,
#             subj.name AS subject,
#             CAST(NULL AS TIMESTAMP) AS screeneddate,
#             TRY_TO_TIMESTAMP(mr.duedate, 'yyyy-MM-dd HH:mm:ss') AS targetdate,
#             CAST(NULL AS STRING) AS amendment,
#             CAST(NULL AS TIMESTAMP) AS amendmentcreateddate,
#             CAST(NULL AS STRING) AS amendmentcreatedby,
#             CAST(NULL AS TIMESTAMP) AS completeddate,
#             CAST(NULL AS STRING) AS completedby,
#             pkg.createdat AS deliverydate,
#             pkg.createdby AS deliveredby,
#             CASE
#                 WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL THEN TRY_TO_TIMESTAMP(mr.closedate, 'yyyy-MM-dd HH:mm:ss')
#             ELSE
#                 TRY_TO_TIMESTAMP(rr.closedate, 'yyyy-MM-dd HH:mm:ss')
#             END AS closeddate,
#             CASE
#                 WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' THEN mr.createdby
#                 WHEN rr.closedate IS NOT NULL AND rr.closedate != 'NULL' THEN rr.createdby
#             ELSE
#                 NULL
#             END AS closedby,
#             rr.notes,
#             CAST(NULL AS STRING) AS withheldstage,
#             CAST(NULL AS STRING) AS otheraddress,
#             CAST(NULL AS STRING) AS holdstage,
#             firstonhold.created_at AS holddate,
#             CAST(NULL AS STRING) AS appealtype,
#             CAST(NULL AS STRING) AS reviewid,
#             CAST(NULL AS TIMESTAMP) AS withhelddate,
#             cr.name AS disposition,
#             CAST(NULL AS STRING) AS execcomments,
#             firstministryrequest.duedate AS originaltargetdate,
#             CAST(NULL AS STRING) AS redactiondescription,
#             CASE
#                 WHEN mr.requeststatuslabel IS NOT NULL THEN mr.requeststatuslabel 
#             ELSE
#                 rr.requeststatuslabel
#             END AS currentactivity,
#             onholddays.total_days_on_hold AS onholddays,
#             onholdbusinessdays.total_business_days_on_hold AS onholdbusinessdays,
#             CASE
#                 WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL AND mr.startdate IS NOT NULL AND TRY_CAST(mr.closedate AS DATE) >= mr.startdate THEN udf_get_workdays_bc(
#                     mr.startdate, -- Start Date
#                     TRY_CAST(mr.closedate AS DATE), -- End Date
#                     (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
#                 ) - COALESCE(onholdbusinessdays.total_business_days_on_hold + 1, 0)
#                 WHEN mr.startdate IS NOT NULL AND CURRENT_DATE() >= mr.startdate THEN udf_get_workdays_bc(
#                     mr.startdate, -- Start Date
#                     CURRENT_DATE(), -- End Date
#                     (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
#                 ) - COALESCE(onholdbusinessdays.total_business_days_on_hold + 1, 0)
#             ELSE
#                 0
#             END AS processeddays,
#             CAST(NULL AS STRING) AS releaseformat,
#             CAST(NULL AS STRING) AS denialauthority,
#             CAST(NULL AS STRING) AS caseowner,
#             CAST(NULL AS STRING) AS caseownertitle,
#             CAST(NULL AS STRING) AS caseowneremail,
#             CAST(NULL AS STRING) AS caseownerphone,
#             firstclosedate.closedate AS originalcloseddate,
#             TRY_TO_TIMESTAMP(mr.duedate, 'yyyy-MM-dd HH:mm:ss') AS stdduedate,
#             CASE
#                 WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate >= mr.closedate THEN udf_get_workdays_bc(
#                     TRY_CAST(mr.closedate AS DATE), -- Start Date
#                     TRY_CAST(mr.duedate AS DATE), -- End Date
#                     (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
#                 )
#                 WHEN mr.duedate IS NOT NULL AND mr.duedate >= CURRENT_DATE() THEN udf_get_workdays_bc(
#                     CURRENT_DATE(), -- Start Date
#                     TRY_CAST(mr.duedate AS DATE), -- End Date
#                     (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
#                 )
#             ELSE
#                 0
#             END AS remainingdays,
#             CAST(NULL AS STRING) AS requestage,
#             TRY_TO_TIMESTAMP(currentactivitydate.created_at, 'yyyy-MM-dd HH:mm:ss') AS currentactivitydate,
#             CAST(NULL AS STRING) AS crossgovtno,
#             mr.description AS requestdescription,
#             CASE
#                 WHEN mr.requeststatusid IS NOT NULL THEN (SELECT name FROM foi_mod.foirequeststatuses WHERE requeststatusid = mr.requeststatusid LIMIT 1) 
#             ELSE
#                 rr.status
#             END AS requeststatus,
#             CASE
#                 WHEN mr.requeststatusid IS NOT NULL THEN (SELECT description FROM foi_mod.foirequeststatuses WHERE requeststatusid = mr.requeststatusid LIMIT 1) 
#             ELSE
#                 rr.status
#             END AS status,
#             CASE
#                 WHEN pkg.createdat IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate >= pkg.createdat THEN 1
#                 WHEN pkg.createdat IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate < pkg.createdat THEN 0
#                 WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' AND mr.duedate IS NOT NULL AND mr.duedate >= mr.closedate THEN 1
#                 WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' AND mr.duedate IS NOT NULL AND mr.duedate < mr.closedate THEN 0
#                 WHEN mr.duedate IS NOT NULL AND mr.duedate >= CURRENT_DATE() THEN 1
#                 WHEN mr.duedate IS NOT NULL AND mr.duedate < CURRENT_DATE() THEN 0
#             ELSE
#                 1
#             END AS countontime,
#             CASE
#                 WHEN pkg.createdat IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate < pkg.createdat THEN 1
#                 WHEN pkg.createdat IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate >= pkg.createdat THEN 0
#                 WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' AND mr.duedate IS NOT NULL AND mr.duedate < mr.closedate THEN 1
#                 WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' AND mr.duedate IS NOT NULL AND mr.duedate >= mr.closedate THEN 0
#                 WHEN mr.duedate IS NOT NULL AND mr.duedate < CURRENT_DATE() THEN 1
#                 WHEN mr.duedate IS NOT NULL AND mr.duedate >= CURRENT_DATE() THEN 0
#             ELSE
#                 0
#             END AS countoverdue,
#             CASE
#                 WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL AND mr.duedate IS NOT NULL AND TRY_CAST(mr.closedate AS DATE) > mr.duedate THEN udf_get_workdays_bc(
#                     TRY_CAST(mr.duedate AS DATE), -- Start Date
#                     TRY_CAST(mr.closedate AS DATE), -- End Date
#                     (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
#                 )
#                 WHEN mr.duedate IS NOT NULL AND CURRENT_DATE() > mr.duedate THEN udf_get_workdays_bc(
#                     TRY_CAST(mr.duedate AS DATE), -- Start Date
#                     CURRENT_DATE(), -- End Date
#                     (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
#                 )
#             ELSE
#                 0
#             END AS daysoverdue,
#             oistatus.name AS publication,
#             oie.name AS publicationreason,
#             latestoipc.oipcno,
#             latestoipc.isjudicialreview AS judicialreview,
#             oipctypes.name AS reviewtype,
#             reason.name AS reason,
#             latestoipc.investigator AS portfolioofficer,
#             CASE
#                 WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL AND mr.duedate IS NOT NULL AND TRY_CAST(mr.closedate AS DATE) > mr.duedate THEN udf_get_workdays_bc(
#                     mr.duedate, -- Start Date
#                     TRY_CAST(mr.closedate AS DATE), -- End Date
#                     (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
#                 )
#                 WHEN mr.duedate IS NOT NULL AND CURRENT_DATE() > mr.duedate THEN udf_get_workdays_bc(
#                     TRY_CAST(mr.duedate AS DATE), -- Start Date
#                     CURRENT_DATE(), -- End Date
#                     (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
#                 )
#             ELSE
#                 0
#             END AS passduedays,
#             mr.description AS description,
#             CAST(NULL AS STRING) AS priority,
#             applicant.foirequestapplicantid AS requesterid,
#             onbehalf.foirequestapplicantid AS onbehalfofrequesterid,
#             CAST(NULL AS STRING) AS referencenumber,
#             CAST(NULL AS STRING) AS refvisualrequestfilenumber,
#             ext.extension,
#             CAST(NULL AS STRING) shipaddressid,
#             CAST(NULL AS STRING) AS billaddressid,
#             TRY_TO_TIMESTAMP(r.receiveddate, 'yyyy-MM-dd HH:mm:ss') AS originalreceiveddate,
#             CAST(NULL AS STRING) AS coordinatednrresponsereqd,
#             CAST(NULL AS STRING) AS applicantfilereference,
#             watchers.watcher_list AS secondaryusers,
#             records.noofdoc AS noofdocdelivered,
#             CASE
#                 WHEN r.isactive = TRUE THEN 'Y'
#             ELSE
#                 'N'
#             END AS activeflag,
#             CAST(NULL AS STRING) AS customfieldstatus,
#             rr.axisrequestid AS visualrequestfilenumber,
#             'FOIMOD' AS sourceoftruth,
#             CASE
#                WHEN mr.assignedto IS NOT NULL THEN ministryassignedto.created_by_when_changed_assignedto
#             ELSE
#                 rawassignedto.created_by_when_changed_assignedto
#             END AS assignedby,
#             CASE
#                 WHEN mr.assignedto IS NOT NULL THEN mr.assignedto 
#             ELSE
#                 rr.assignedto
#             END AS assignedto
#         FROM foi_mod.foirequests r
#         INNER JOIN (
#             SELECT
#                 foirequestid,
#                 MAX(version) as version
#             FROM foi_mod.foirequests
#             GROUP BY foirequestid
#         ) max_request ON r.foirequestid = max_request.foirequestid AND r.version = max_request.version
#         LEFT JOIN foi_mod.foiministryrequests mr ON r.foirequestid = mr.foirequest_id AND r.version = mr.foirequestversion_id
#         INNER JOIN foi_mod.foirawrequests rr ON r.foirawrequestid = rr.requestid
#         INNER JOIN (
#             SELECT
#                 requestid,
#                 MAX(version) as version
#             FROM foi_mod.foirawrequests
#             GROUP BY requestid
#         ) max_rawrequest ON rr.requestid = max_rawrequest.requestid AND rr.version = max_rawrequest.version
#         INNER JOIN (
#             SELECT
#                 requestid,
#                 TRY_CAST(created_at AS DATE) AS created_at
#             FROM foi_mod.foirawrequests
#             WHERE version = 1
#         ) first_rawrequest ON r.foirawrequestid = first_rawrequest.requestid
#         LEFT JOIN (
#             WITH LatestAssignedTo AS (
#                 SELECT
#                     foirequest_id,
#                     assignedto AS current_assignedto_value
#                 FROM
#                     foi_mod.foiministryrequests
#                     QUALIFY ROW_NUMBER() OVER (
#                         PARTITION BY foirequest_id ORDER BY version DESC
#                     ) = 1
#             ),
#             FirstOccurrenceOfCurrentAssignment AS (
#                 SELECT
#                     t.foirequest_id,
#                     TRY_CAST(t.created_at AS DATE) AS time_assignedto_changed_to_current_value,
#                     t.createdby AS created_by_when_changed_assignedto
#                 FROM
#                     foi_mod.foiministryrequests AS t
#                 INNER JOIN
#                     LatestAssignedTo AS lat ON t.foirequest_id = lat.foirequest_id AND t.assignedto = lat.current_assignedto_value
#                     QUALIFY ROW_NUMBER() OVER (
#                         PARTITION BY t.foirequest_id ORDER BY t.version ASC
#                     ) = 1
#             )
#             SELECT
#                 lat.foirequest_id,
#                 lat.current_assignedto_value AS latest_assignedto,
#                 fo.time_assignedto_changed_to_current_value,
#                 fo.created_by_when_changed_assignedto
#             FROM LatestAssignedTo AS lat
#             INNER JOIN FirstOccurrenceOfCurrentAssignment AS fo
#             ON lat.foirequest_id = fo.foirequest_id
#             ORDER BY lat.foirequest_id
#         ) ministryassignedto on ministryassignedto.foirequest_id = mr.foirequest_id
#         INNER JOIN (
#             WITH LatestAssignedTo AS (
#                 SELECT
#                     requestid,
#                     assignedto AS current_assignedto_value
#                 FROM
#                     foi_mod.foirawrequests
#                     QUALIFY ROW_NUMBER() OVER (
#                         PARTITION BY requestid ORDER BY version DESC
#                     ) = 1
#             ),
#             FirstOccurrenceOfCurrentAssignment AS (
#                 SELECT
#                     t.requestid,
#                     TRY_CAST(t.created_at AS DATE) AS time_assignedto_changed_to_current_value,
#                     t.createdby AS created_by_when_changed_assignedto
#                 FROM
#                     foi_mod.foirawrequests AS t
#                 INNER JOIN
#                     LatestAssignedTo AS lat ON t.requestid = lat.requestid AND t.assignedto = lat.current_assignedto_value
#                     QUALIFY ROW_NUMBER() OVER (
#                         PARTITION BY t.requestid ORDER BY t.version ASC
#                     ) = 1
#             )
#             SELECT
#                 lat.requestid,
#                 lat.current_assignedto_value AS latest_assignedto,
#                 fo.time_assignedto_changed_to_current_value,
#                 fo.created_by_when_changed_assignedto
#             FROM LatestAssignedTo AS lat
#             INNER JOIN FirstOccurrenceOfCurrentAssignment AS fo
#             ON lat.requestid = fo.requestid
#             ORDER BY lat.requestid
#         ) rawassignedto on rawassignedto.requestid = rr.requestid
#         LEFT JOIN foi_mod.foiministryrequestsubjectcodes msubj on msubj.foiministryrequestid = mr.foiministryrequestid AND msubj.foiministryrequestversion = mr.version
#         LEFT JOIN foi_mod.subjectcodes subj on subj.subjectcodeid = msubj.subjectcodeid
#         LEFT JOIN (
#             SELECT
#                 ministryrequestid,
#                 TRY_CAST(createdat AS DATE) AS createdat,
#                 createdby
#             FROM docreviewer.pdfstitchpackage
#             QUALIFY ROW_NUMBER() OVER (PARTITION BY ministryrequestid ORDER BY pdfstitchpackageid DESC) = 1
#         ) pkg on pkg.ministryrequestid = mr.foiministryrequestid
#         LEFT JOIN (
#             SELECT
#                 foiministryrequestid,
#                 TRY_CAST(created_at AS DATE) AS created_at
#             FROM foi_mod.foiministryrequests
#             WHERE requeststatuslabel = 'onhold'
#             QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version ASC) = 1
#         ) firstonhold on firstonhold.foiministryrequestid = mr.foiministryrequestid
#         LEFT JOIN (
#             SELECT
#                 foiministryrequest_id,
#                 foiministryrequestversion_id,
#                 oipcno,
#                 isjudicialreview,
#                 investigator,
#                 reviewtypeid,
#                 reasonid
#             FROM foi_mod.foirequestoipc
#             QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequest_id ORDER BY foiministryrequestversion_id DESC) = 1
#         ) latestoipc on latestoipc.foiministryrequest_id = mr.foiministryrequestid AND latestoipc.foiministryrequestversion_id = mr.version
#         --LEFT JOIN foi_mod.foirequestoipc oipc on oipc.foiministryrequest_id = mr.foiministryrequestid AND oipc.foiministryrequestversion_id = mr.version
#         LEFT JOIN foi_mod.oipcreviewtypes oipctypes on oipctypes.reviewtypeid = latestoipc.reviewtypeid
#         LEFT JOIN foi_mod.oipcreasons reason on reason.reasonid = latestoipc.reasonid
#         INNER JOIN foi_mod.foirequestapplicantmappings applicant on applicant.foirequest_id = r.foirequestid AND applicant.foirequestversion_id = r.version AND applicant.requestortypeid = 1
#         LEFT JOIN foi_mod.foirequestapplicantmappings onbehalf on onbehalf.foirequest_id = r.foirequestid AND onbehalf.foirequestversion_id = r.version AND onbehalf.requestortypeid = 2
#         LEFT JOIN (
#             SELECT
#                 foiministryrequestid,
#                 TRY_CAST(created_at AS DATE) AS created_at
#             FROM foi_mod.foiministryrequests
#             WHERE duedate IS NULL
#             QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version ASC) = 1
#         ) firstduedate on firstduedate.foiministryrequestid = mr.foiministryrequestid
#         LEFT JOIN (
#             SELECT
#                 foiministryrequestid,
#                 TRY_CAST(duedate AS DATE) AS duedate
#             FROM foi_mod.foiministryrequests
#             WHERE version = 1
#         ) firstministryrequest on firstministryrequest.foiministryrequestid = mr.foiministryrequestid
#         LEFT JOIN (
#             SELECT
#                 foiministryrequestid,
#                 TRY_CAST(closedate AS DATE) AS closedate
#             FROM foi_mod.foiministryrequests
#             WHERE closedate IS NOT NULL AND closedate != 'NULL'
#             QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version ASC) = 1
#         ) firstclosedate on firstclosedate.foiministryrequestid = mr.foiministryrequestid
#         LEFT JOIN (
#             WITH LatestStatus AS (
#                 SELECT
#                     foiministryrequestid,
#                     requeststatuslabel AS status
#                 FROM
#                     foi_mod.foiministryrequests
#                     QUALIFY ROW_NUMBER() OVER (
#                         PARTITION BY foiministryrequestid ORDER BY version DESC
#                     ) = 1
#             )
#             SELECT
#                 fmr.foiministryrequestid,
#                 fmr.created_at
#             FROM foi_mod.foiministryrequests fmr
#             INNER JOIN LatestStatus ON fmr.foiministryrequestid = LatestStatus.foiministryrequestid AND fmr.requeststatuslabel = LatestStatus.status
#             QUALIFY ROW_NUMBER() OVER (PARTITION BY fmr.foiministryrequestid ORDER BY fmr.version ASC) = 1
#         ) currentactivitydate on currentactivitydate.foiministryrequestid = mr.foiministryrequestid
#         LEFT JOIN (
#             SELECT
#                 ministryrequestid,
#                 ministryrequestversion,
#                 count(recordid) AS noofdoc
#             FROM foi_mod.foirequestrecords
#             WHERE isactive = 't'
#             GROUP BY ministryrequestid, ministryrequestversion
#         ) records ON records.ministryrequestid = mr.foiministryrequestid AND records.ministryrequestversion = mr.version
#         LEFT JOIN (
#             SELECT
#                 ministryrequestid,
#                 ARRAY_JOIN(COLLECT_SET(watchedby), ', ') AS watcher_list
#             FROM foi_mod.foirequestwatchers
#             WHERE isactive = 't'
#             GROUP BY ministryrequestid
#         ) watchers ON watchers.ministryrequestid = mr.foiministryrequestid
#         LEFT JOIN (
#             SELECT
#                 foiministryrequest_id,
#                 foiministryrequestversion_id,
#                 SUM(COALESCE(TRY_CAST(approvednoofdays AS INT), 0)) AS extension
#             FROM foi_mod.foirequestextensions
#             WHERE isactive = 't'
#             GROUP BY foiministryrequest_id, foiministryrequestversion_id
#         ) ext ON ext.foiministryrequest_id = mr.foiministryrequestid AND ext.foiministryrequestversion_id = mr.version
#         LEFT JOIN (
#             SELECT
#                 foiministryrequestid,
#                 SUM(days_in_status) AS total_days_on_hold
#             FROM (
#                 SELECT
#                     foiministryrequestid,
#                     requeststatusid,
#                     MIN(created_at) AS status_start_date,
#                     -- The end date of a status period is the created_at of the next different status version
#                     -- Or CURRENT_DATE() if it's the latest status for that foiministryrequestid
#                     COALESCE(
#                       LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
#                       CURRENT_DATE() -- Use CURRENT_DATE() to include time in current status
#                     ) AS status_end_date,
#                     COALESCE(
#                       DATEDIFF(LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)), MIN(created_at)),
#                       DATEDIFF(CURRENT_DATE(), MIN(created_at))
#                     ) AS days_in_status
#                 FROM (
#                     SELECT
#                       foiministryrequestid,
#                       version,
#                       requeststatusid,
#                       created_at,
#                       -- Create a running sum to identify consecutive status blocks
#                       SUM(CASE WHEN LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) != requeststatusid OR LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) IS NULL THEN 1 ELSE 0 END)
#                         OVER (PARTITION BY foiministryrequestid ORDER BY version) AS status_group_id
#                     FROM
#                       foi_mod.foiministryrequests
#                 ) AS sub_inner
#                 GROUP BY
#                     foiministryrequestid,
#                     requeststatusid,
#                     status_group_id
#             ) AS sub_outer
#             WHERE
#                 requeststatusid = 11 --'On Hold'
#             GROUP BY
#                 foiministryrequestid
#         ) onholddays ON onholddays.foiministryrequestid = mr.foiministryrequestid
#         LEFT JOIN (
#             SELECT
#                 foiministryrequestid,
#                 SUM(days_in_status) AS total_business_days_on_hold
#             FROM (
#                 SELECT
#                     foiministryrequestid,
#                     requeststatusid,
#                     MIN(created_at) AS status_start_date,
#                     -- The end date of a status period is the created_at of the next different status version
#                     -- Or CURRENT_DATE() if it's the latest status for that foiministryrequestid
#                     COALESCE(
#                       LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
#                       CURRENT_DATE() -- Use CURRENT_DATE() to include time in current status
#                     ) AS status_end_date,
#                     udf_get_workdays_bc(
#                       MIN(created_at), -- On hold start date
#                       COALESCE(
#                         LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
#                         CURRENT_DATE()
#                       ), -- On hold start date
#                       (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
#                     )-1 AS days_in_status
#                 FROM (
#                     SELECT
#                       foiministryrequestid,
#                       version,
#                       requeststatusid,
#                       created_at,
#                       -- Create a running sum to identify consecutive status blocks
#                       SUM(CASE WHEN LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) != requeststatusid OR LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) IS NULL THEN 1 ELSE 0 END)
#                         OVER (PARTITION BY foiministryrequestid ORDER BY version) AS status_group_id
#                     FROM
#                       foi_mod.foiministryrequests
#                 ) AS sub_inner
#                 GROUP BY
#                     foiministryrequestid,
#                     requeststatusid,
#                     status_group_id
#             ) AS sub_outer
#             WHERE
#                 requeststatusid = 11 --'On Hold'
#             GROUP BY
#                 foiministryrequestid
#         ) onholdbusinessdays ON onholdbusinessdays.foiministryrequestid = mr.foiministryrequestid
#         LEFT JOIN foi_mod.programareas office on office.programareaid = mr.programareaid
#         LEFT JOIN (
#             SELECT
#                 foiministryrequest_id,
#                 oiexemption_id,
#                 oipublicationstatus_id
#             FROM
#                 foi_mod.foiopeninformationrequests
#                 QUALIFY ROW_NUMBER() OVER (
#                     PARTITION BY foiministryrequest_id ORDER BY version DESC
#                 ) = 1
#         ) oi ON oi.foiministryrequest_id = mr.foiministryrequestid
#         LEFT JOIN foi_mod.openinfopublicationstatuses oistatus ON oistatus.oipublicationstatusid = oi.oipublicationstatus_id
#         LEFT JOIN foi_mod.openinformationexemptions oie ON oie.oiexemptionid = oi.oiexemption_id
#         LEFT JOIN foi_mod.closereasons cr ON try_cast(cr.closereasonid as bigint) = try_cast(mr.closereasonid as bigint)
#         WHERE r.created_at > '2025-05-01'

In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers
from pyspark.sql.functions import lit, col

tablename = "factrequestdetails"
runcycleid = etl_helpers.start_run_cycle(tablename)
os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:

    df_lastrun = spark.sql(f"SELECT runcyclestartat as createddate FROM dimruncycle WHERE packagename = \"{tablename}\" AND success = 't' ORDER BY runcycleid DESC LIMIT 1")
    
    if df_lastrun.count() > 0:
        lastruntime = df_lastrun.first().createddate.strftime("%Y-%m-%d %H:%M:%S")
    else:
        lastruntime = "2019-01-01 00:00:00"
    print(lastruntime)

    query = f"""
        SELECT
            {runcycleid} as runcycleid,
            mr.foirequest_id as foirequestid,
            CASE
                WHEN r.requesttype = 'personal' THEN 33
            ELSE
                31
            END AS requesttypeid,
            CASE
                WHEN mr.requeststatusid IS NOT NULL THEN mr.requeststatusid
            ELSE
                (SELECT requeststatusid FROM foi_mod.foirequeststatuses WHERE name = rr.status LIMIT 1)
            END AS requeststatusid,
            TRY_CAST(r.receivedmodeid AS STRING) AS receivedmodeid,
            TRY_CAST(r.deliverymodeid AS STRING) AS deliverymodeid,
            r.applicantcategoryid,
            TRY_TO_TIMESTAMP(r.receiveddate, 'yyyy-MM-dd HH:mm:ss') AS requesteddate,
            TRY_TO_TIMESTAMP(r.receiveddate, 'yyyy-MM-dd HH:mm:ss') AS receiveddate,
            TRY_TO_TIMESTAMP(mr.startdate, 'yyyy-MM-dd HH:mm:ss') AS startdate,
            TRY_TO_TIMESTAMP(first_rawrequest.created_at, 'yyyy-MM-dd HH:mm:ss') AS createddate,
            TRY_TO_TIMESTAMP(r.created_at, 'yyyy-MM-dd HH:mm:ss') AS modifieddate,
            CASE
               WHEN mr.assignedto IS NOT NULL THEN TRY_TO_TIMESTAMP(ministryassignedto.time_assignedto_changed_to_current_value, 'yyyy-MM-dd HH:mm:ss')
            ELSE
                TRY_TO_TIMESTAMP(rawassignedto.time_assignedto_changed_to_current_value, 'yyyy-MM-dd HH:mm:ss')
            END AS assigneddate,
            CAST(NULL AS STRING) AS receivedbyusername,
            r.createdby AS modifiedbyusername,
            CAST(NULL AS INTEGER) AS assignedbyid,
            CAST(NULL AS INTEGER) AS assignedtoid,
            CASE
                WHEN mr.assignedto IS NOT NULL THEN mr.assignedto 
            ELSE
                rr.assignedto
            END AS primaryusername,
            0 AS primarygroupid,
            CASE
                WHEN mr.assignedgroup IS NOT NULL THEN mr.assignedgroup 
            ELSE
                rr.assignedgroup
            END AS groupname,
            office.bcgovcode AS officeid,
            mr.assignedministrygroup AS ministry,
            subj.name AS subject,
            CAST(NULL AS TIMESTAMP) AS screeneddate,
            TRY_CAST(mr.duedate AS STRING) AS targetdate,
            CAST(NULL AS STRING) AS amendment,
            CAST(NULL AS TIMESTAMP) AS amendmentcreateddate,
            CAST(NULL AS STRING) AS amendmentcreatedby,
            CAST(NULL AS TIMESTAMP) AS completeddate,
            CAST(NULL AS STRING) AS completedby,
            TRY_TO_TIMESTAMP(pkg.createdat, 'yyyy-MM-dd HH:mm:ss') AS deliverydate,
            pkg.createdby AS deliveredby,
            CASE
                WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL THEN TRY_TO_TIMESTAMP(mr.closedate, 'yyyy-MM-dd HH:mm:ss')
            ELSE
                TRY_TO_TIMESTAMP(rr.closedate, 'yyyy-MM-dd HH:mm:ss')
            END AS closeddate,
            CASE
                WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' THEN mr.createdby
                WHEN rr.closedate IS NOT NULL AND rr.closedate != 'NULL' THEN rr.createdby
            ELSE
                NULL
            END AS closedby,
            rr.notes,
            CAST(NULL AS STRING) AS withheldstage,
            CAST(NULL AS STRING) AS otheraddress,
            CAST(NULL AS STRING) AS holdstage,
            TRY_TO_TIMESTAMP(firstonhold.created_at, 'yyyy-MM-dd HH:mm:ss') AS holddate,
            CAST(NULL AS STRING) AS appealtype,
            CAST(NULL AS STRING) AS reviewid,
            CAST(NULL AS TIMESTAMP) AS withhelddate,
            cr.name AS disposition,
            CAST(NULL AS STRING) AS execcomments,
            TRY_TO_TIMESTAMP(firstministryrequest.duedate, 'yyyy-MM-dd HH:mm:ss') AS originaltargetdate,
            CAST(NULL AS STRING) AS redactiondescription,
            CASE
                WHEN mr.requeststatuslabel IS NOT NULL THEN mr.requeststatuslabel 
            ELSE
                rr.requeststatuslabel
            END AS currentactivity,
            TRY_CAST(onholddays.total_days_on_hold AS INTEGER) AS onholddays,
            TRY_CAST(onholdbusinessdays.total_business_days_on_hold AS INTEGER) AS onholdbusinessdays,
            TRY_CAST(
            CASE
                WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL AND mr.startdate IS NOT NULL AND TRY_CAST(mr.closedate AS DATE) >= mr.startdate THEN udf_get_workdays_bc(
                    mr.startdate, -- Start Date
                    TRY_CAST(mr.closedate AS DATE), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays)
                ) - COALESCE(onholdbusinessdays.total_business_days_on_hold + 1, 0)
                WHEN mr.startdate IS NOT NULL AND CURRENT_DATE() >= mr.startdate THEN udf_get_workdays_bc(
                    mr.startdate, -- Start Date
                    CURRENT_DATE(), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays)
                ) - COALESCE(onholdbusinessdays.total_business_days_on_hold + 1, 0)
            ELSE
                0
            END
            AS INTEGER
            ) AS processeddays,
            CAST(NULL AS STRING) AS releaseformat,
            CAST(NULL AS STRING) AS denialauthority,
            CAST(NULL AS STRING) AS caseowner,
            CAST(NULL AS STRING) AS caseownertitle,
            CAST(NULL AS STRING) AS caseowneremail,
            CAST(NULL AS STRING) AS caseownerphone,
            TRY_TO_TIMESTAMP(firstclosedate.closedate, 'yyyy-MM-dd HH:mm:ss') AS originalcloseddate,
            TRY_TO_TIMESTAMP(mr.duedate, 'yyyy-MM-dd HH:mm:ss') AS stdduedate,
            TRY_CAST(
            CASE
                WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate >= mr.closedate THEN udf_get_workdays_bc(
                    TRY_CAST(mr.closedate AS DATE), -- Start Date
                    TRY_CAST(mr.duedate AS DATE), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays)
                )
                WHEN mr.duedate IS NOT NULL AND mr.duedate >= CURRENT_DATE() THEN udf_get_workdays_bc(
                    CURRENT_DATE(), -- Start Date
                    TRY_CAST(mr.duedate AS DATE), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays)
                )
            ELSE
                0
            END
            AS INTEGER
            ) AS remainingdays,
            TRY_CAST(NULL AS INTEGER) AS requestage,
            TRY_TO_TIMESTAMP(currentactivitydate.created_at, 'yyyy-MM-dd HH:mm:ss') AS currentactivitydate,
            CAST(NULL AS STRING) AS crossgovtno,
            mr.description AS requestdescription,
            CASE
                WHEN mr.requeststatusid IS NOT NULL THEN (SELECT name FROM foi_mod.foirequeststatuses WHERE requeststatusid = mr.requeststatusid LIMIT 1) 
            ELSE
                rr.status
            END AS requeststatus,
            CASE
                WHEN mr.requeststatusid IS NOT NULL THEN (SELECT description FROM foi_mod.foirequeststatuses WHERE requeststatusid = mr.requeststatusid LIMIT 1) 
            ELSE
                rr.status
            END AS status,
            CASE
                WHEN pkg.createdat IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate >= pkg.createdat THEN 1
                WHEN pkg.createdat IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate < pkg.createdat THEN 0
                WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' AND mr.duedate IS NOT NULL AND mr.duedate >= mr.closedate THEN 1
                WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' AND mr.duedate IS NOT NULL AND mr.duedate < mr.closedate THEN 0
                WHEN mr.duedate IS NOT NULL AND mr.duedate >= CURRENT_DATE() THEN 1
                WHEN mr.duedate IS NOT NULL AND mr.duedate < CURRENT_DATE() THEN 0
            ELSE
                1
            END AS countontime,
            CASE
                WHEN pkg.createdat IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate < pkg.createdat THEN 1
                WHEN pkg.createdat IS NOT NULL AND mr.duedate IS NOT NULL AND mr.duedate >= pkg.createdat THEN 0
                WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' AND mr.duedate IS NOT NULL AND mr.duedate < mr.closedate THEN 1
                WHEN mr.closedate IS NOT NULL AND mr.closedate != 'NULL' AND mr.duedate IS NOT NULL AND mr.duedate >= mr.closedate THEN 0
                WHEN mr.duedate IS NOT NULL AND mr.duedate < CURRENT_DATE() THEN 1
                WHEN mr.duedate IS NOT NULL AND mr.duedate >= CURRENT_DATE() THEN 0
            ELSE
                0
            END AS countoverdue,
            CASE
                WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL AND mr.duedate IS NOT NULL AND TRY_CAST(mr.closedate AS DATE) > mr.duedate THEN udf_get_workdays_bc(
                    TRY_CAST(mr.duedate AS DATE), -- Start Date
                    TRY_CAST(mr.closedate AS DATE), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays)
                )
                WHEN mr.duedate IS NOT NULL AND CURRENT_DATE() > mr.duedate THEN udf_get_workdays_bc(
                    TRY_CAST(mr.duedate AS DATE), -- Start Date
                    CURRENT_DATE(), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays)
                )
            ELSE
                0
            END AS daysoverdue,
            oistatus.name AS publication,
            oie.name AS publicationreason,
            latestoipc.oipcno,
            latestoipc.isjudicialreview AS judicialreview,
            oipctypes.name AS reviewtype,
            reason.name AS reason,
            latestoipc.investigator AS portfolioofficer,
            TRY_CAST(
            CASE
                WHEN TRY_CAST(mr.closedate AS DATE) IS NOT NULL AND mr.duedate IS NOT NULL AND TRY_CAST(mr.closedate AS DATE) > mr.duedate THEN udf_get_workdays_bc(
                    mr.duedate, -- Start Date
                    TRY_CAST(mr.closedate AS DATE), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays)
                )
                WHEN mr.duedate IS NOT NULL AND CURRENT_DATE() > mr.duedate THEN udf_get_workdays_bc(
                    TRY_CAST(mr.duedate AS DATE), -- Start Date
                    CURRENT_DATE(), -- End Date
                    (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays)
                )
            ELSE
                0
            END
            AS INTEGER
            ) AS passduedays,
            mr.description AS description,
            CAST(NULL AS STRING) AS priority,
            TRY_CAST(applicant.foirequestapplicantid AS INTEGER) AS requesterid,
            TRY_CAST(onbehalf.foirequestapplicantid AS STRING) AS onbehalfofrequesterid,
            CAST(NULL AS STRING) AS referencenumber,
            CAST(NULL AS STRING) AS refvisualrequestfilenumber,
            TRY_CAST(ext.extension AS STRING) AS extension,
            CAST(NULL AS STRING) shipaddressid,
            CAST(NULL AS STRING) AS billaddressid,
            TRY_TO_TIMESTAMP(r.receiveddate, 'yyyy-MM-dd HH:mm:ss') AS originalreceiveddate,
            CAST(NULL AS STRING) AS coordinatednrresponsereqd,
            CAST(NULL AS STRING) AS applicantfilereference,
            watchers.watcher_list AS secondaryusers,
            TRY_CAST(records.noofdoc AS INTEGER) AS noofdocdelivered,
            CASE
                WHEN r.isactive = TRUE THEN 'Y'
            ELSE
                'N'
            END AS activeflag,
            CAST(NULL AS STRING) AS customfieldstatus,
            rr.axisrequestid AS visualrequestfilenumber,
            'FOIMOD' AS sourceoftruth,
            CASE
               WHEN mr.assignedto IS NOT NULL THEN ministryassignedto.created_by_when_changed_assignedto
            ELSE
                rawassignedto.created_by_when_changed_assignedto
            END AS assignedby,
            CASE
                WHEN mr.assignedto IS NOT NULL THEN mr.assignedto 
            ELSE
                rr.assignedto
            END AS assignedto
        FROM foi_mod.foirequests r
        INNER JOIN (
            SELECT
                foirequestid,
                MAX(version) as version
            FROM foi_mod.foirequests
            GROUP BY foirequestid
        ) max_request ON r.foirequestid = max_request.foirequestid AND r.version = max_request.version
        LEFT JOIN foi_mod.foiministryrequests mr ON r.foirequestid = mr.foirequest_id AND r.version = mr.foirequestversion_id
        INNER JOIN foi_mod.foirawrequests rr ON r.foirawrequestid = rr.requestid
        INNER JOIN (
            SELECT
                requestid,
                MAX(version) as version
            FROM foi_mod.foirawrequests
            GROUP BY requestid
        ) max_rawrequest ON rr.requestid = max_rawrequest.requestid AND rr.version = max_rawrequest.version
        INNER JOIN (
            SELECT
                requestid,
                TRY_CAST(created_at AS DATE) AS created_at
            FROM foi_mod.foirawrequests
            WHERE version = 1
        ) first_rawrequest ON r.foirawrequestid = first_rawrequest.requestid
        LEFT JOIN (
            WITH LatestAssignedTo AS (
                SELECT
                    foirequest_id,
                    assignedto AS current_assignedto_value
                FROM
                    foi_mod.foiministryrequests
                    QUALIFY ROW_NUMBER() OVER (
                        PARTITION BY foirequest_id ORDER BY version DESC
                    ) = 1
            ),
            FirstOccurrenceOfCurrentAssignment AS (
                SELECT
                    t.foirequest_id,
                    TRY_CAST(t.created_at AS DATE) AS time_assignedto_changed_to_current_value,
                    t.createdby AS created_by_when_changed_assignedto
                FROM
                    foi_mod.foiministryrequests AS t
                INNER JOIN
                    LatestAssignedTo AS lat ON t.foirequest_id = lat.foirequest_id AND t.assignedto = lat.current_assignedto_value
                    QUALIFY ROW_NUMBER() OVER (
                        PARTITION BY t.foirequest_id ORDER BY t.version ASC
                    ) = 1
            )
            SELECT
                lat.foirequest_id,
                lat.current_assignedto_value AS latest_assignedto,
                fo.time_assignedto_changed_to_current_value,
                fo.created_by_when_changed_assignedto
            FROM LatestAssignedTo AS lat
            INNER JOIN FirstOccurrenceOfCurrentAssignment AS fo
            ON lat.foirequest_id = fo.foirequest_id
            ORDER BY lat.foirequest_id
        ) ministryassignedto on ministryassignedto.foirequest_id = mr.foirequest_id
        INNER JOIN (
            WITH LatestAssignedTo AS (
                SELECT
                    requestid,
                    assignedto AS current_assignedto_value
                FROM
                    foi_mod.foirawrequests
                    QUALIFY ROW_NUMBER() OVER (
                        PARTITION BY requestid ORDER BY version DESC
                    ) = 1
            ),
            FirstOccurrenceOfCurrentAssignment AS (
                SELECT
                    t.requestid,
                    TRY_CAST(t.created_at AS DATE) AS time_assignedto_changed_to_current_value,
                    t.createdby AS created_by_when_changed_assignedto
                FROM
                    foi_mod.foirawrequests AS t
                INNER JOIN
                    LatestAssignedTo AS lat ON t.requestid = lat.requestid AND t.assignedto = lat.current_assignedto_value
                    QUALIFY ROW_NUMBER() OVER (
                        PARTITION BY t.requestid ORDER BY t.version ASC
                    ) = 1
            )
            SELECT
                lat.requestid,
                lat.current_assignedto_value AS latest_assignedto,
                fo.time_assignedto_changed_to_current_value,
                fo.created_by_when_changed_assignedto
            FROM LatestAssignedTo AS lat
            INNER JOIN FirstOccurrenceOfCurrentAssignment AS fo
            ON lat.requestid = fo.requestid
            ORDER BY lat.requestid
        ) rawassignedto on rawassignedto.requestid = rr.requestid
        LEFT JOIN foi_mod.foiministryrequestsubjectcodes msubj on msubj.foiministryrequestid = mr.foiministryrequestid AND msubj.foiministryrequestversion = mr.version
        LEFT JOIN foi_mod.subjectcodes subj on subj.subjectcodeid = msubj.subjectcodeid
        LEFT JOIN (
            SELECT
                ministryrequestid,
                TRY_CAST(createdat AS DATE) AS createdat,
                createdby
            FROM docreviewer.pdfstitchpackage
            QUALIFY ROW_NUMBER() OVER (PARTITION BY ministryrequestid ORDER BY pdfstitchpackageid DESC) = 1
        ) pkg on pkg.ministryrequestid = mr.foiministryrequestid
        LEFT JOIN (
            SELECT
                foiministryrequestid,
                TRY_CAST(created_at AS DATE) AS created_at
            FROM foi_mod.foiministryrequests
            WHERE requeststatuslabel = 'onhold'
            QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version ASC) = 1
        ) firstonhold on firstonhold.foiministryrequestid = mr.foiministryrequestid
        LEFT JOIN (
            SELECT
                foiministryrequest_id,
                foiministryrequestversion_id,
                oipcno,
                isjudicialreview,
                investigator,
                reviewtypeid,
                reasonid
            FROM foi_mod.foirequestoipc
            QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequest_id ORDER BY foiministryrequestversion_id DESC) = 1
        ) latestoipc on latestoipc.foiministryrequest_id = mr.foiministryrequestid AND latestoipc.foiministryrequestversion_id = mr.version
        --LEFT JOIN foi_mod.foirequestoipc oipc on oipc.foiministryrequest_id = mr.foiministryrequestid AND oipc.foiministryrequestversion_id = mr.version
        LEFT JOIN foi_mod.oipcreviewtypes oipctypes on oipctypes.reviewtypeid = latestoipc.reviewtypeid
        LEFT JOIN foi_mod.oipcreasons reason on reason.reasonid = latestoipc.reasonid
        INNER JOIN foi_mod.foirequestapplicantmappings applicant on applicant.foirequest_id = r.foirequestid AND applicant.foirequestversion_id = r.version AND applicant.requestortypeid = 1
        LEFT JOIN foi_mod.foirequestapplicantmappings onbehalf on onbehalf.foirequest_id = r.foirequestid AND onbehalf.foirequestversion_id = r.version AND onbehalf.requestortypeid = 2
        LEFT JOIN (
            SELECT
                foiministryrequestid,
                TRY_CAST(created_at AS DATE) AS created_at
            FROM foi_mod.foiministryrequests
            WHERE duedate IS NULL
            QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version ASC) = 1
        ) firstduedate on firstduedate.foiministryrequestid = mr.foiministryrequestid
        LEFT JOIN (
            SELECT
                foiministryrequestid,
                TRY_CAST(duedate AS DATE) AS duedate
            FROM foi_mod.foiministryrequests
            WHERE version = 1
        ) firstministryrequest on firstministryrequest.foiministryrequestid = mr.foiministryrequestid
        LEFT JOIN (
            SELECT
                foiministryrequestid,
                TRY_CAST(closedate AS DATE) AS closedate
            FROM foi_mod.foiministryrequests
            WHERE closedate IS NOT NULL AND closedate != 'NULL'
            QUALIFY ROW_NUMBER() OVER (PARTITION BY foiministryrequestid ORDER BY version ASC) = 1
        ) firstclosedate on firstclosedate.foiministryrequestid = mr.foiministryrequestid
        LEFT JOIN (
            WITH LatestStatus AS (
                SELECT
                    foiministryrequestid,
                    requeststatuslabel AS status
                FROM
                    foi_mod.foiministryrequests
                    QUALIFY ROW_NUMBER() OVER (
                        PARTITION BY foiministryrequestid ORDER BY version DESC
                    ) = 1
            )
            SELECT
                fmr.foiministryrequestid,
                fmr.created_at
            FROM foi_mod.foiministryrequests fmr
            INNER JOIN LatestStatus ON fmr.foiministryrequestid = LatestStatus.foiministryrequestid AND fmr.requeststatuslabel = LatestStatus.status
            QUALIFY ROW_NUMBER() OVER (PARTITION BY fmr.foiministryrequestid ORDER BY fmr.version ASC) = 1
        ) currentactivitydate on currentactivitydate.foiministryrequestid = mr.foiministryrequestid
        LEFT JOIN (
            SELECT
                ministryrequestid,
                ministryrequestversion,
                count(recordid) AS noofdoc
            FROM foi_mod.foirequestrecords
            WHERE isactive = 't'
            GROUP BY ministryrequestid, ministryrequestversion
        ) records ON records.ministryrequestid = mr.foiministryrequestid AND records.ministryrequestversion = mr.version
        LEFT JOIN (
            SELECT
                ministryrequestid,
                ARRAY_JOIN(COLLECT_SET(watchedby), ', ') AS watcher_list
            FROM foi_mod.foirequestwatchers
            WHERE isactive = 't'
            GROUP BY ministryrequestid
        ) watchers ON watchers.ministryrequestid = mr.foiministryrequestid
        LEFT JOIN (
            SELECT
                foiministryrequest_id,
                foiministryrequestversion_id,
                SUM(COALESCE(TRY_CAST(approvednoofdays AS INT), 0)) AS extension
            FROM foi_mod.foirequestextensions
            WHERE isactive = 't'
            GROUP BY foiministryrequest_id, foiministryrequestversion_id
        ) ext ON ext.foiministryrequest_id = mr.foiministryrequestid AND ext.foiministryrequestversion_id = mr.version
        LEFT JOIN (
            SELECT
                foiministryrequestid,
                SUM(days_in_status) AS total_days_on_hold
            FROM (
                SELECT
                    foiministryrequestid,
                    requeststatusid,
                    MIN(created_at) AS status_start_date,
                    -- The end date of a status period is the created_at of the next different status version
                    -- Or CURRENT_DATE() if it's the latest status for that foiministryrequestid
                    COALESCE(
                      LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
                      CURRENT_DATE() -- Use CURRENT_DATE() to include time in current status
                    ) AS status_end_date,
                    COALESCE(
                      DATEDIFF(LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)), MIN(created_at)),
                      DATEDIFF(CURRENT_DATE(), MIN(created_at))
                    ) AS days_in_status
                FROM (
                    SELECT
                      foiministryrequestid,
                      version,
                      requeststatusid,
                      created_at,
                      -- Create a running sum to identify consecutive status blocks
                      SUM(CASE WHEN LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) != requeststatusid OR LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) IS NULL THEN 1 ELSE 0 END)
                        OVER (PARTITION BY foiministryrequestid ORDER BY version) AS status_group_id
                    FROM
                      foi_mod.foiministryrequests
                ) AS sub_inner
                GROUP BY
                    foiministryrequestid,
                    requeststatusid,
                    status_group_id
            ) AS sub_outer
            WHERE
                requeststatusid = 11 --'On Hold'
            GROUP BY
                foiministryrequestid
        ) onholddays ON onholddays.foiministryrequestid = mr.foiministryrequestid
        LEFT JOIN (
            SELECT
                foiministryrequestid,
                SUM(days_in_status) AS total_business_days_on_hold
            FROM (
                SELECT
                    foiministryrequestid,
                    requeststatusid,
                    MIN(created_at) AS status_start_date,
                    -- The end date of a status period is the created_at of the next different status version
                    -- Or CURRENT_DATE() if it's the latest status for that foiministryrequestid
                    COALESCE(
                      LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
                      CURRENT_DATE() -- Use CURRENT_DATE() to include time in current status
                    ) AS status_end_date,
                    udf_get_workdays_bc(
                      MIN(created_at), -- On hold start date
                      COALESCE(
                        LEAD(MIN(created_at)) OVER (PARTITION BY foiministryrequestid ORDER BY MIN(created_at)),
                        CURRENT_DATE()
                      ), -- On hold start date
                      (SELECT COLLECT_LIST(holiday_date) FROM default.bc_holidays) -- Pass the array of BC holidays
                    )-1 AS days_in_status
                FROM (
                    SELECT
                      foiministryrequestid,
                      version,
                      requeststatusid,
                      created_at,
                      -- Create a running sum to identify consecutive status blocks
                      SUM(CASE WHEN LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) != requeststatusid OR LAG(requeststatusid) OVER (PARTITION BY foiministryrequestid ORDER BY version) IS NULL THEN 1 ELSE 0 END)
                        OVER (PARTITION BY foiministryrequestid ORDER BY version) AS status_group_id
                    FROM
                      foi_mod.foiministryrequests
                ) AS sub_inner
                GROUP BY
                    foiministryrequestid,
                    requeststatusid,
                    status_group_id
            ) AS sub_outer
            WHERE
                requeststatusid = 11 --'On Hold'
            GROUP BY
                foiministryrequestid
        ) onholdbusinessdays ON onholdbusinessdays.foiministryrequestid = mr.foiministryrequestid
        LEFT JOIN foi_mod.programareas office on office.programareaid = mr.programareaid
        LEFT JOIN (
            SELECT
                foiministryrequest_id,
                oiexemption_id,
                oipublicationstatus_id
            FROM
                foi_mod.foiopeninformationrequests
                QUALIFY ROW_NUMBER() OVER (
                    PARTITION BY foiministryrequest_id ORDER BY version DESC
                ) = 1
        ) oi ON oi.foiministryrequest_id = mr.foiministryrequestid
        LEFT JOIN foi_mod.openinfopublicationstatuses oistatus ON oistatus.oipublicationstatusid = oi.oipublicationstatus_id
        LEFT JOIN foi_mod.openinformationexemptions oie ON oie.oiexemptionid = oi.oiexemption_id
        LEFT JOIN foi_mod.closereasons cr ON try_cast(cr.closereasonid as bigint) = try_cast(mr.closereasonid as bigint)
        WHERE r.created_at > '{lastruntime}'
    """

    # print(query)

    df = spark.sql(query)
    df.show()

    # order of columns here is important!
    df_mapped = df.selectExpr(
        "runcycleid AS runcycleid",
        "foirequestid AS foirequestid",
        "requesttypeid AS requesttypeid",
        "requeststatusid AS requeststatusid",
        "receivedmodeid AS receivedmodeid",
        "deliverymodeid AS deliverymodeid",
        "applicantcategoryid AS applicantcategoryid",
        "requesteddate AS requesteddate",
        "receiveddate AS receiveddate",
        "startdate AS startdate",
        "createddate AS createddate",
        "modifieddate AS modifieddate",
        "assigneddate AS assigneddate",
        "receivedbyusername AS receivedbyusername",
        "modifiedbyusername AS modifiedbyusername",
        "assignedbyid AS assignedbyid",
        "assignedtoid AS assignedtoid",
        "primaryusername AS primaryusername",
        "primarygroupid AS primarygroupid",
        "groupname AS groupname",
        "officeid AS officeid",
        "ministry AS ministry",
        "subject AS subject",
        "screeneddate AS screeneddate",
        "targetdate AS targetdate",
        "amendment AS amendment",
        "amendmentcreateddate AS amendmentcreateddate",
        "amendmentcreatedby AS amendmentcreatedby",
        "completeddate AS completeddate",
        "completedby AS completedby",
        "deliverydate AS deliverydate",
        "deliveredby AS deliveredby",
        "closeddate AS closeddate",
        "closedby AS closedby",
        "notes AS notes",
        "withheldstage AS withheldstage",
        "otheraddress AS otheraddress",
        "holdstage AS holdstage",
        "holddate AS holddate",
        "appealtype AS appealtype",
        "reviewid AS reviewid",
        "withhelddate AS withhelddate",
        "disposition AS disposition",
        "execcomments AS execcomments",
        "originaltargetdate AS originaltargetdate",
        "redactiondescription AS redactiondescription",
        "currentactivity AS currentactivity",
        "onholddays AS onholddays",
        "onholdbusinessdays AS onholdbusinessdays",
        "processeddays AS processeddays",
        "releaseformat AS releaseformat",
        "denialauthority AS denialauthority",
        "caseowner AS caseowner",
        "caseownertitle AS caseownertitle",
        "caseowneremail AS caseowneremail",
        "caseownerphone AS caseownerphone",
        "originalcloseddate AS originalcloseddate",
        "stdduedate AS stdduedate",
        "remainingdays AS remainingdays",
        "requestage AS requestage",
        "currentactivitydate AS currentactivitydate",
        "crossgovtno AS crossgovtno",
        "requestdescription AS requestdescription",
        "requeststatus AS requeststatus",
        "status AS status",
        "countontime AS countontime",
        "countoverdue AS countoverdue",
        "daysoverdue AS daysoverdue",
        "publication AS publication",
        "publicationreason AS publicationreason",
        "oipcno AS oipcno",
        "judicialreview AS judicialreview",
        "reviewtype AS reviewtype",
        "reason AS reason",
        "portfolioofficer AS portfolioofficer",
        "passduedays AS passduedays",
        "description AS description",
        "priority AS priority",
        "requesterid AS requesterid",
        "onbehalfofrequesterid AS onbehalfofrequesterid",
        "referencenumber AS referencenumber",
        "refvisualrequestfilenumber AS refvisualrequestfilenumber",
        "extension AS extension",
        "shipaddressid AS shipaddressid",
        "billaddressid AS billaddressid",
        "originalreceiveddate AS originalreceiveddate",
        "coordinatednrresponsereqd AS coordinatednrresponsereqd",
        "applicantfilereference AS applicantfilereference",
        "secondaryusers AS secondaryusers",
        "noofdocdelivered AS noofdocdelivered",
        "activeflag AS activeflag",
        "customfieldstatus AS customfieldstatus",
        "visualrequestfilenumber AS visualrequestfilenumber",
        "sourceoftruth AS sourceoftruth",
        "assignedby AS assignedby",
        "assignedto AS assignedto"
    )
    df_mapped.show()

    from delta.tables import DeltaTable
    delta_table = DeltaTable.forName(spark, f"hive_metastore.default.{tablename}")
    delta_table.alias("target").merge(
        df_mapped.alias("source"),
        "target.foirequestid = source.foirequestid AND target.sourceoftruth = source.sourceoftruth"
    ).whenMatchedUpdate(
        condition = "target.activeflag = 'Y'",
        set = {
            "activeflag": lit("N"),
        }
    ).execute()

    print("Matched records deactivated.")

    df_mapped.write.format("delta").mode("append").saveAsTable(f"hive_metastore.default.{tablename}") 

    # ).whenNotMatchedInsert(values = {
    #     "runcycleid": col("source.runcycleid"),
    #     "foirequestid": col("source.foirequestid"),
    #     "requesttypeid": col("source.requesttypeid"),
    #     "requeststatusid": col("source.requeststatusid"),
    #     "receivedmodeid": col("source.receivedmodeid"),
    #     "deliverymodeid": col("source.deliverymodeid"),
    #     "applicantcategoryid": col("source.applicantcategoryid"),
    #     "requesteddate": col("source.requesteddate"),
    #     "receiveddate": col("source.receiveddate"),
    #     "startdate": col("source.startdate"),
    #     "createddate": col("source.createddate"),
    #     "modifieddate": col("source.modifieddate"),
    #     "assigneddate": col("source.assigneddate"),
    #     "receivedbyusername": col("source.receivedbyusername"),
    #     "modifiedbyusername": col("source.modifiedbyusername"),
    #     "assignedbyid": col("source.assignedbyid"),
    #     "assignedtoid": col("source.assignedtoid"),
    #     "primaryusername": col("source.primaryusername"),
    #     "primarygroupid": col("source.primarygroupid"),
    #     "groupname": col("source.groupname"),
    #     "officeid": col("source.officeid"),
    #     "ministry": col("source.ministry"),
    #     "subject": col("source.subject"),
    #     "screeneddate": col("source.screeneddate"),
    #     "targetdate": col("source.targetdate"),
    #     "amendment": col("source.amendment"),
    #     "amendmentcreateddate": col("source.amendmentcreateddate"),
    #     "amendmentcreatedby": col("source.amendmentcreatedby"),
    #     "completeddate": col("source.completeddate"),
    #     "completedby": col("source.completedby"),
    #     "deliverydate": col("source.deliverydate"),
    #     "deliveredby": col("source.deliveredby"),
    #     "closeddate": col("source.closeddate"),
    #     "closedby": col("source.closedby"),
    #     "notes": col("source.notes"),
    #     "withheldstage": col("source.withheldstage"),
    #     "otheraddress": col("source.otheraddress"),
    #     "holdstage": col("source.holdstage"),
    #     "holddate": col("source.holddate"),
    #     "appealtype": col("source.appealtype"),
    #     "reviewid": col("source.reviewid"),
    #     "withhelddate": col("source.withhelddate"),
    #     "disposition": col("source.disposition"),
    #     "execcomments": col("source.execcomments"),
    #     "originaltargetdate": col("source.originaltargetdate"),
    #     "redactiondescription": col("source.redactiondescription"),
    #     "currentactivity": col("source.currentactivity"),
    #     "onholddays": col("source.onholddays"),
    #     "onholdbusinessdays": col("source.onholdbusinessdays"),
    #     "processeddays": col("source.processeddays"),
    #     "releaseformat": col("source.releaseformat"),
    #     "denialauthority": col("source.denialauthority"),
    #     "caseowner": col("source.caseowner"),
    #     "caseownertitle": col("source.caseownertitle"),
    #     "caseowneremail": col("source.caseowneremail"),
    #     "caseownerphone": col("source.caseownerphone"),
    #     "originalcloseddate": col("source.originalcloseddate"),
    #     "stdduedate": col("source.stdduedate"),
    #     "remainingdays": col("source.remainingdays"),
    #     "requestage": col("source.requestage"),
    #     "currentactivitydate": col("source.currentactivitydate"),
    #     "crossgovtno": col("source.crossgovtno"),
    #     "requestdescription": col("source.requestdescription"),
    #     "requeststatus": col("source.requeststatus"),
    #     "status": col("source.status"),
    #     "countontime": col("source.countontime"),
    #     "countoverdue": col("source.countoverdue"),
    #     "daysoverdue": col("source.daysoverdue"),
    #     "publication": col("source.publication"),
    #     "publicationreason": col("source.publicationreason"),
    #     "oipcno": col("source.oipcno"),
    #     "judicialreview": col("source.judicialreview"),
    #     "reviewtype": col("source.reviewtype"),
    #     "reason": col("source.reason"),
    #     "portfolioofficer": col("source.portfolioofficer"),
    #     "passduedays": col("source.passduedays"),
    #     "description": col("source.description"),
    #     "priority": col("source.priority"),
    #     "requesterid": col("source.requesterid"),
    #     "onbehalfofrequesterid": col("source.onbehalfofrequesterid"),
    #     "referencenumber": col("source.referencenumber"),
    #     "refvisualrequestfilenumber": col("source.refvisualrequestfilenumber"),
    #     "extension": col("source.extension"),
    #     "shipaddressid": col("source.shipaddressid"),
    #     "billaddressid": col("source.billaddressid"),
    #     "originalreceiveddate": col("source.originalreceiveddate"),
    #     "coordinatednrresponsereqd": col("source.coordinatednrresponsereqd"),
    #     "applicantfilereference": col("source.applicantfilereference"),
    #     "secondaryusers": col("source.secondaryusers"),
    #     "noofdocdelivered": col("source.noofdocdelivered"),
    #     "activeflag": col("source.activeflag"),
    #     "customfieldstatus": col("source.customfieldstatus"),
    #     "visualrequestfilenumber": col("source.visualrequestfilenumber"),
    #     "sourceoftruth": col("source.sourceoftruth"),
    #     "assignedby": col("source.assignedby"),
    #     "assignedto": col("source.assignedto")
    # }).execute()

    etl_helpers.end_run_cycle(runcycleid, 't', tablename)
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', tablename, "Credentials not available")
    raise Exception("notebook failed") from e
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', tablename)
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', tablename, f"An error occurred: {e}")
        raise Exception("notebook failed") from e